<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="font-family:newtimeroman;color:red;font-size:200%;text-align:left;border-radius:9px 9px;">CONTENTS</p>   
    
* [1. Introduction](#1)
    
* [2. Importing](#2)
    
* [3. Loading Dataset](#3)
    
* [4. Data Outline](#4)
    
    * [4.1. Basic Information](#4.1)    
    * [4.2. Description of Dataset](#4.2)    
    * [4.3. Change Column Names](#4.3)    
    * [4.4. Other Basic Information about Dataset](#4.4)
    * [4.5. Correlation between Variables](#4.5)
    
    
* [5. Visualization](#5)
    
    * [5.1. Heatmap](#5.1)    
    * [5.2. Pairplot](#5.2)    
    * [5.3. Distplot](#5.3)    
    * [5.4. Lineplot](#5.4)
    * [5.5. Jointplot](#5.5)
    * [5.6. Plotting 'close' price history](#5.6)
    * [5.7. Histogram with Plotly Express](#5.7)
    
    
    
* [6. Data Preprocessing](#6)
    
    * [6.1. New Dataframe with only 'close' Column](#6.1)    
    * [6.2. Converting New Dataframe to Array](#6.2)    
    * [6.3. The number of Rows to train the Model on](#6.3)    
    * [6.4. Normalization](#6.4)
    * [6.5. Creating Training dataset](#6.5)
    * [6.6. Convert x_train and y_train to array](#6.6)
    * [6.7. Reshaping the data](#6.7)
   
    
* [7. LONG SHORT-TERM MEMORY (LSTM) Model](#7)
    
    * [7.1. Building and Training LSTM model](#7.1)    
    * [7.2. Create testing dataset](#7.2)    
    * [7.3. Convert test data to array and reshape it](#7.3)    
    * [7.4. Get predicted prices](#7.4)
    * [7.5. Get root mean squared error (RMSE)](#7.5)
    * [7.6. Plot the data](#7.6)
    * [7.7. Show the valid and predicted prices](#7.7)

<a id="1"></a>

## <b>1 <span style='color:#F4D940'>|</span> Introduction</b>

<a id="2"></a>

## <b>2 <span style='color:#F4D940'>|</span> Importing</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import figure_factory as figfac

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import math
from sklearn.metrics import mean_squared_error
#!pip install keras-tuner --upgrade
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# to ignore warnings
import warnings
warnings.filterwarnings("ignore")

<a id="1"></a>

## <b>3 <span style='color:#F4D940'>|</span> Loading Dataset</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
WCT = pd.read_csv("../input/worldcall-telecom-limited/WTL.csv")

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
df = WCT.copy()
df.reset_index(drop=True, inplace=True)
df = df.drop('ID',axis=1)
fig = figfac.create_table(df.head(n = 15))
fig.show()

<a id="4"></a>

## <b>4 <span style='color:#F4D940'>|</span> Data Outline</b>

<a id="4.1"></a>

#### <b>4.1 <span style='color:#F4D940'>|</span> Basic Information</b>

In [ ]:
df.info()

<a id="4.2"></a>

#### <b>4.2 <span style='color:#F4D940'>|</span> Description of Dataset</b>

In [ ]:
df.describe().T

<a id="4.3"></a>

#### <b>4.3 <span style='color:#F4D940'>|</span> Change Column Names</b>

In [ ]:
df.rename(columns = {"Open" : "open", "High" : "high", "Low" : "low",
                     "Close" : "close", "Volume" : "volume"}, inplace = True)

<a id="4.4"></a>

#### <b>4.4 <span style='color:#F4D940'>|</span> Other Basic Information about Dataset</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
print("Shape of the dataset: ", df.shape)
print("Length of the dataset: ", len(df))
print("Maximum closing price of stock during last 4.5 years: ", df["close"].max())
print("Minimum closing price of stock during last 4.5 years: ", df["close"].min())

<a id="4.5"></a>

#### <b>4.5 <span style='color:#F4D940'>|</span> Correlation between Variables</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
corr = df.corr()
corr.style.background_gradient(cmap = "gist_stern")

<a id="5"></a>

## <b>5 <span style='color:#F4D940'>|</span> Visualization</b>

<a id="5.1"></a>

#### <b>5.1<span style='color:#F4D940'>|</span> Heatmap</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
plt.figure(figsize = [10, 5], clear = True, facecolor = 'grey')
sns.heatmap(df.corr(), annot = True, square = False, linewidths = 2,
            linecolor = "white", cmap = "magma");

<a id="5.2"></a>

#### <b>5.2<span style='color:#F4D940'>|</span> Pairplot</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
sns.pairplot(df, diag_kind = "reg", height = 2, aspect = 1, corner = True);

<a id="5.3"></a>

#### <b>5.3<span style='color:#F4D940'>|</span> Distplot</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
fig, axes = plt.subplots(1, 3, figsize=(20, 5))
sns.distplot(ax = axes[0], x = df["open"], hist = True, bins = 10, kde = True,
             vertical = False, color = "y").set(title = "open price distribution");

sns.distplot(ax = axes[1], x = df["high"], hist = True, bins = 10, kde = True,
             vertical = False, color = "g").set(title = "high price distribution");

sns.distplot(ax = axes[2], x = df["low"], hist = True, bins = 10, kde = True,
             vertical = False, color = "b").set(title = "low price distribution");

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
fig, axes = plt.subplots(1, 2, figsize=(20, 5))

sns.distplot(ax = axes[0], x = df["close"], hist = True, bins = 10, kde = True,
             vertical = False, color = "r").set(title = "closing price distribution");

sns.distplot(ax = axes[1], x = df["volume"], hist = True, bins = 10, kde = True,
             vertical = False, color = "black").set(title = "volume distribution");

<a id="5.4"></a>

#### <b>5.4<span style='color:#F4D940'>|</span> Lineplot</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
plt.style.use('fivethirtyeight')
fig, axes = plt.subplots(1, 2, figsize=(20, 5))

sns.lineplot(ax = axes[0], x = df["date"], y = df["close"]);
sns.lineplot(ax = axes[1], x = df["date"], y = df["volume"]);

<a id="5.5"></a>

#### <b>5.5<span style='color:#F4D940'>|</span> Jointplot</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
sns.jointplot(x = "close", y = "low", data = df, height = 5, ratio = 6, kind = "scatter");

<a id="5.6"></a>

#### <b>5.6<span style='color:#F4D940'>|</span> Plotting 'close' price history</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
plt.figure(figsize = (17, 7))
plt.title("close price history")
plt.plot(df["close"])
plt.xlabel("date", fontsize = 15)
plt.ylabel("close price", fontsize = 15)
plt.show()

<a id="5.7"></a>
​
#### <b>5.7<span style='color:#F4D940'>|</span> Histogram with Plotly Express</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
fig = px.histogram(df, x = "close",
                   y = "volume",
                   marginal = "box",
                   hover_data  = df.columns)
fig.show()

<a id="6"></a>

## <b>6 <span style='color:#F4D940'>|</span> Data Preprocessing</b>

<a id="6.1"></a>

#### <b>6.1<span style='color:#F4D940'>|</span> New Dataframe with 'close' Column</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
data = df.filter(['close'])
data.head()

<a id="6.2"></a>

#### <b>6.2<span style='color:#F4D940'>|</span> Converting New Dataframe to Array</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
dataset = data.values
dataset[0:5]

<a id="6.3"></a>

#### <b>6.3<span style='color:#F4D940'>|</span> The number of Rows to train the Model on</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
training_data_len = int(np.ceil( len(dataset) * .95 ))
training_data_len

<a id="6.4"></a>

#### <b>6.4<span style='color:#F4D940'>|</span> Normalization</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

<a id="6.5"></a>

#### <b>6.5<span style='color:#F4D940'>|</span> Creating Training dataset</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
# Create the training data set 
# Create the scaled training data set
train_data = scaled_data[0:int(training_data_len), :]
# Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)

<a id="6.6"></a>

#### <b>6.6<span style='color:#F4D940'>|</span> Convert x_train and y_train to array</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
x_train = np.array(x_train)
y_train = np.array(y_train)

<a id="6.7"></a>

#### <b>6.7<span style='color:#F4D940'>|</span> Reshaping the data</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

<a id="7"></a>

## <b>7 <span style='color:#F4D940'>|</span> LONG SHORT-TERM MEMORY (LSTM) Model</b>

<a id="7.1"></a>

#### <b>7.1<span style='color:#F4D940'>|</span> Building and Training LSTM model</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(128, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer = "adam", loss = "mean_squared_error")

model.fit(x_train, y_train, batch_size=1, epochs=1)

<a id="7.2"></a>

#### <b>7.2<span style='color:#F4D940'>|</span> Create testing dataset</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
# Create the testing data set
# Create a new array containing scaled values from index 1543 to 2002 
test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

<a id="7.3"></a>

#### <b>7.3<span style='color:#F4D940'>|</span> Convert test data to array and reshape it</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

<a id="7.4"></a>

#### <b>7.4<span style='color:#F4D940'>|</span> Get predicted prices</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
predictions[0:10]

<a id="7.5"></a>

#### <b>7.5<span style='color:#F4D940'>|</span> Get root mean squared error (RMSE)</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
rmse

<a id="7.6"></a>

#### <b>7.6<span style='color:#F4D940'>|</span> Plot the data</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
# Visualize the data
plt.figure(figsize=(16,6))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price', fontsize=18)
plt.plot(train['close'])
plt.plot(valid[['close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

<a id="7.7"></a>

#### <b>7.7<span style='color:#F4D940'>|</span> Show the valid and predicted prices</b>

In [ ]:
#code by https://www.kaggle.com/code/eisgandar/gold-price-prediction-with-lstm/notebook
valid.head(n = 10)

Thank you !